In [ ]:
# [HBASE]
# nohup ./hbase thrift start &
# sudo firewall-cmd --permanent --zone=public  -add-port 9090/tcp
# sudo sudo firewall-cmd --reload

In [1]:
import happybase
import pandas as pd

In [2]:
env = 'pro'

In [3]:
hbase_host_dict = dict()
hbase_host_dict['hbase_host_uat'] = '10.88.3.54' # uat
hbase_host_dict['hbase_host_pro'] = '10.88.6.243' # pro

In [4]:
host = 'hbase_host_' + env
connection = happybase.Connection(hbase_host_dict[host])

In [5]:
table_list = list(set([table.decode('utf-8')[:-9]for table in connection.tables()]))

In [6]:
table_list

['tb_view_time',
 'recom_response_result',
 'tb_cart_time',
 'recom_click_result',
 'log_by_kafka_topic']

In [9]:
def generate_hbase_value(v):
    value_set = list()
    for key, value in v.items():
        value = value.decode('utf-8').replace("|","")
        value_set.append(value)
    return '|'.join([element for element in value_set])

In [10]:
for date in range(20190120, 20190121):
    date = str(date)
    for i in range(len(table_list)):
        table_name = '-'.join([table_list[i],date])
        table = connection.table(table_name)
        file_path = '/'.join(['files', env, date, table_name])
        with open (file_path,'w') as f:
            a = list()
            for k, v in table.scan():
                key_list = [(k.decode('utf-8')) for k in v.keys()]
                value_str = generate_hbase_value(v)
                a.append(value_str)
            columns = '|'.join([column for column in key_list])
            f.write(columns + '\n')
            for l in a:
                f.write(l + '\n')
    print("Finished {0}".format(date))

Finished 20190120
